In [1]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf



***************************************************************
Failed to import TensorFlow. Please note that TensorFlow is not installed by default when you install TFDS. This allow you to choose to install either `tf-nightly` or `tensorflow`. Please install the most recent version of TensorFlow, by following instructions at https://tensorflow.org/install.
***************************************************************




ModuleNotFoundError: No module named 'tensorflow'

In [2]:
pip install --upgrade pip

  Using cached pip-21.1.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import rouge
import nltk

In [3]:
import os
#from gtts import gTTS 
from pathlib import Path
import appdirs
import gdown
import torch
import logging
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from summarizer import Summarizer
#from summarizer import TransformerSummarizer

In [4]:
class BartSumSummarizer():
    def __init__(self, state_dict_key='model', pretrained="facebook/bart-large-cnn"):
        bart = BartForConditionalGeneration.from_pretrained(pretrained)
        tokenizer = BartTokenizer.from_pretrained(pretrained)
        self.tokenizer = tokenizer
        self.bart = bart

    def summarize_string(self, source_line, min_length, max_length):
        source_line = [source_line]
        inputs = self.tokenizer.batch_encode_plus(source_line, truncation=True, return_tensors='pt')
        # Generate Summary
        summary_ids = self.bart.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], num_beams=4, min_length=min_length, max_length=max_length)
        return [self.tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
       

In [5]:
def bart(contents):
    document = str(contents)
    doc_length = len(document.split())
    min_length = int(doc_length/6)
    max_length = min_length+400
    transcript_summarized = BartSumSummarizer().summarize_string(document, min_length=min_length, max_length=max_length)
    return transcript_summarized


In [6]:
import sklearn.metrics as metric
def get_score(all_hypothesis, all_references):
    apply_avg = 'Avg'
    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                               max_n=4,
                               limit_length=True,
                               length_limit=100,
                               length_limit_type='words',
                               apply_avg=apply_avg,
                               alpha=0.5, # Default F1_score
                               weight_factor=1.2,
                               stemming=True)
    scores = evaluator.get_scores(all_hypothesis, all_references)
    #print(scores)
    for key,value in scores.items():
        print(key.upper()+'\n')
        print("F: "+str(value['f'])+"  R: "+str(value['r'])+"  P: "+str(value['p'])+"\n")
    #print(scores)

In [7]:
def get_summaries(model,folder,start,end):
    all_hypothesis = []
    all_references = []
    for i in range(start,end):
          print(i)
          num=str(i)
          if(len(num)==1):
                num='0'+'0'+num
          elif(len(num)==2):
                num='0'+num
          t = open("/Users/shreyabanerjee/Summarizer/BBC News Summary/News Articles/"+folder+"/"+num+".txt", "r")
          text=t.read()
          l= open("/Users/shreyabanerjee/Summarizer/BBC News Summary/Summaries/"+folder+"/"+num+".txt", "r")
          label=l.read()
          if(label!=''):
              summary=model(text)    #replace bart function name with bert or gpt2 for other models
              #print("Text\n",text,'\n\nSummary\n',label,'\n\n My Summary\n',summary)
              all_hypothesis.append(summary)
              final= open("/Users/shreyabanerjee/Summarizer/BBC News Summary/Bart_Summaries/"+folder+"/"+num+".txt", "w")
              final.write(summary)
              all_references.append(label)
    return all_hypothesis,all_references

In [9]:
candidate_bart_business,reference_bart_business=get_summaries(bart,"business",1,11)
candidate_bart_entertainment,reference_bart_entertainment=get_summaries(bart,"entertainment",1,11)
candidate_bart_sport,reference_bart_sport=get_summaries(bart,"sport",1,11)
candidate_bart_politics,reference_bart_politics=get_summaries(bart,"politics",1,11)
candidate_bart_tech,reference_bart_tech=get_summaries(bart,"tech",1,11)

1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10


In [10]:
get_score(candidate_bart_business,reference_bart_business)

ROUGE-3

F: 0.13481056170812977  R: 0.09542778578453896  P: 0.23848192164229895

ROUGE-2

F: 0.18157272890794093  R: 0.13000939501728198  P: 0.31094732242609574

ROUGE-4

F: 0.11246797860283722  R: 0.07891769198279433  P: 0.20400646459451877

ROUGE-1

F: 0.3489400836299487  R: 0.2527753419389789  P: 0.5785094805039693

ROUGE-L

F: 0.29035902009702647  R: 0.21800011864493687  P: 0.4438735875443678

ROUGE-W

F: 0.08923507641881458  R: 0.052341349593974164  P: 0.3154586097567964



In [11]:
get_score(candidate_bart_entertainment,reference_bart_entertainment)

ROUGE-3

F: 0.24147125361652028  R: 0.1723056128540277  P: 0.4102501469166516

ROUGE-2

F: 0.2901664943094978  R: 0.20725853414913184  P: 0.49095324133925544

ROUGE-4

F: 0.21301873497927032  R: 0.15170678523178194  P: 0.3640671377500022

ROUGE-1

F: 0.417433333027388  R: 0.297893317226392  P: 0.706440172362444

ROUGE-L

F: 0.3977893331037378  R: 0.2964018262964262  P: 0.6102542554379169

ROUGE-W

F: 0.13086344455226348  R: 0.07667973172674641  P: 0.4539546020558009



In [12]:
get_score(candidate_bart_sport,reference_bart_sport)

ROUGE-3

F: 0.20017228202945242  R: 0.15575624819047756  P: 0.2968338282389952

ROUGE-2

F: 0.24383838696635599  R: 0.1890767884587325  P: 0.36310636570007665

ROUGE-4

F: 0.17343394487906322  R: 0.1356425901114463  P: 0.255026255785333

ROUGE-1

F: 0.39257503049039033  R: 0.3018724200219043  P: 0.5968210702368751

ROUGE-L

F: 0.32591249979274617  R: 0.2587641204793757  P: 0.45975283481395524

ROUGE-W

F: 0.11332317210685186  R: 0.07005295172080739  P: 0.3305949649157859



In [13]:
get_score(candidate_bart_politics,reference_bart_politics)

ROUGE-3

F: 0.13999472050956024  R: 0.10925258964605752  P: 0.2011250006281366

ROUGE-2

F: 0.18706842446820973  R: 0.14805294371446562  P: 0.26242297721934665

ROUGE-4

F: 0.12172876708958924  R: 0.09438221894483656  P: 0.17725442474187095

ROUGE-1

F: 0.3771723059886203  R: 0.30527254628669726  P: 0.5088674335490557

ROUGE-L

F: 0.3031647915665693  R: 0.25154968969081565  P: 0.3900979545518714

ROUGE-W

F: 0.09614347704592467  R: 0.05954503548934176  P: 0.26021890838816847



In [15]:
get_score(candidate_bart_tech,reference_bart_tech)

ROUGE-3

F: 0.1484366978693647  R: 0.1317316574180214  P: 0.17795548020482865

ROUGE-2

F: 0.20380429463166036  R: 0.17880245484882856  P: 0.24950662492568704

ROUGE-4

F: 0.12896842730814756  R: 0.1148476612207453  P: 0.15370541958807588

ROUGE-1

F: 0.42477827046408995  R: 0.3704440116468289  P: 0.5220098516106784

ROUGE-L

F: 0.3327439610164262  R: 0.2952966268218698  P: 0.39368909719374434

ROUGE-W

F: 0.11672382241707073  R: 0.07597772016787252  P: 0.2668707951505146

